In [134]:
import pandas as pd
import os
from openai import OpenAI

## 데이터 불러오기

In [135]:
df = pd.read_excel('/Users/hj/Documents/무제/자연어 명령어 SQL 쿼리 생성/AdidasSalesdata.xlsx')
df

,Retailer,Retailer ID,Invoice Date,Region,State,City,Gender Type,Product Category,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
0,Foot Locker,1185732,2021-10-26,Northeast,Pennsylvania,Philadelphia,Men,Apparel,55,125,68750.0,24062.50,0.35,Outlet
1,Foot Locker,1185732,2021-10-27,Northeast,Pennsylvania,Philadelphia,Women,Apparel,45,225,101250.0,30375.00,0.30,Outlet
2,Foot Locker,1185732,2021-10-28,Northeast,Pennsylvania,Philadelphia,Men,Street Footwear,45,475,213750.0,117562.50,0.55,Outlet
3,Foot Locker,1185732,2021-10-29,Northeast,Pennsylvania,Philadelphia,Men,Athletic Footwear,45,125,56250.0,19687.50,0.35,Outlet
4,Foot Locker,1185732,2021-10-30,Northeast,Pennsylvania,Philadelphia,Women,Street Footwear,35,175,61250.0,24500.00,0.40,Outlet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9643,West Gear,1128299,2020-03-14,West,Nevada,Las Vegas,Women,Apparel,56,170,9520.0,1713.60,0.18,Outlet
9644,West Gear,1128299,2020-03-15,West,Nevada,Las Vegas,Men,Street Footwear,20,149,2980.0,1192.00,0.40,Outlet
9645,West Gear,1128299,2020-03-16,West,Nevada,Las Vegas,Men,Athletic Footwear,31,145,4495.0,1123.75,0.25,Outlet
9646,West Gear,1128299,2020-03-17,West,Nevada,Las Vegas,Women,Street Footwear,26,128,3328.0,1397.76,0.42,Outlet


In [136]:
df.columns

Index(['Retailer', 'Retailer ID', 'Invoice Date', 'Region', 'State', 'City',
       'Gender Type', 'Product Category', 'Price per Unit', 'Units Sold',
       'Total Sales', 'Operating Profit', 'Operating Margin', 'Sales Method'],
      dtype='object')

In [137]:
df.columns = [col.replace(' ','_') for col in df.columns]
df.columns

Index(['Retailer', 'Retailer_ID', 'Invoice_Date', 'Region', 'State', 'City',
       'Gender_Type', 'Product_Category', 'Price_per_Unit', 'Units_Sold',
       'Total_Sales', 'Operating_Profit', 'Operating_Margin', 'Sales_Method'],
      dtype='object')

In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9648 entries, 0 to 9647
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Retailer          9648 non-null   object        
 1   Retailer_ID       9648 non-null   int64         
 2   Invoice_Date      9648 non-null   datetime64[ns]
 3   Region            9648 non-null   object        
 4   State             9648 non-null   object        
 5   City              9648 non-null   object        
 6   Gender_Type       9648 non-null   object        
 7   Product_Category  9648 non-null   object        
 8   Price_per_Unit    9648 non-null   int64         
 9   Units_Sold        9648 non-null   int64         
 10  Total_Sales       9648 non-null   float64       
 11  Operating_Profit  9648 non-null   float64       
 12  Operating_Margin  9648 non-null   float64       
 13  Sales_Method      9648 non-null   object        
dtypes: datetime64[ns](1), fl

## 데이터 확인하기

In [139]:
# 각 region 별 sum
df.groupby("Region")['Total_Sales'].sum().sort_values(ascending=False)

Region
West         269943182.0
Northeast    186324067.0
Southeast    163171236.0
South        144663181.0
Midwest      135800459.0
Name: Total_Sales, dtype: float64

## SQL Database 셋팅하기

In [140]:
from sqlalchemy import create_engine  # 임시 데이터베이스 만듦
from sqlalchemy import text

In [141]:
temp_db = create_engine('sqlite:///:memory:', echo=True)

In [142]:
data = df.to_sql(name='Sales', con=temp_db)  ## con = connection

2024-06-21 15:09:38,231 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-21 15:09:38,234 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2024-06-21 15:09:38,234 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-21 15:09:38,235 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2024-06-21 15:09:38,235 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-21 15:09:38,236 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"Retailer" TEXT, 
	"Retailer_ID" BIGINT, 
	"Invoice_Date" DATETIME, 
	"Region" TEXT, 
	"State" TEXT, 
	"City" TEXT, 
	"Gender_Type" TEXT, 
	"Product_Category" TEXT, 
	"Price_per_Unit" BIGINT, 
	"Units_Sold" BIGINT, 
	"Total_Sales" FLOAT, 
	"Operating_Profit" FLOAT, 
	"Operating_Margin" FLOAT, 
	"Sales_Method" TEXT
)


2024-06-21 15:09:38,237 INFO sqlalchemy.engine.Engine [no key 0.00034s] ()
2024-06-21 15:09:38,238 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Sales_index" ON "Sales" ("index")
2024-06-21 15:09:38,238 IN

In [143]:
## python으로 쿼리 작성하는 방법

with temp_db.connect() as conn:
    result = conn.execute(text("SELECT Retailer, City, State, Total_Sales from Sales ORDER BY Total_Sales DESC LIMIT 1"))

2024-06-21 15:09:38,325 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-21 15:09:38,325 INFO sqlalchemy.engine.Engine SELECT Retailer, City, State, Total_Sales from Sales ORDER BY Total_Sales DESC LIMIT 1
2024-06-21 15:09:38,326 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ()
2024-06-21 15:09:38,327 INFO sqlalchemy.engine.Engine ROLLBACK


In [144]:
result.all()

[('Walmart', 'Orlando', 'Florida', 825000.0)]

## OPEN AI 셋팅
### API 사용을 위한 Key 입력 및 셋팅

In [145]:
api_key = os.getenv('OPENAI_API_KEY')

In [146]:
client = OpenAI()

In [147]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [148]:
# GPT 한테 우리가 다루는 데이터, 테이블이 어떤 구조인지 알려주는 함수 (테이블 컬럼들 뭐있는지)

def tabel_definition_prompt(df):
    prompt = '''Given the following SQL definition, write queries based on the request
                \n### sqlite SQL table, with its properties:
    #
    # Sales({})
    # 
    '''.format(",".join(str(x) for x in df.columns))
    
    return prompt

In [149]:
print(tabel_definition_prompt(df))

Given the following SQL definition, write queries based on the request
                
### sqlite SQL table, with its properties:
    #
    # Sales(Retailer,Retailer_ID,Invoice_Date,Region,State,City,Gender_Type,Product_Category,Price_per_Unit,Units_Sold,Total_Sales,Operating_Profit,Operating_Margin,Sales_Method)
    # 
    


In [150]:
# 사용자로부터 어떤걸 확인하고 싶은지 ? 받는 내용의 함수 

def prompt_input():
    nlp_text = input("질의 하고자 하는 내용을 입력해주세요: ")
    return nlp_text

In [151]:
nlp_text = prompt_input()

In [152]:
full_prompt = str(tabel_definition_prompt(df)) + str(nlp_text)
full_prompt

'Given the following SQL definition, write queries based on the request\n                \n### sqlite SQL table, with its properties:\n    #\n    # Sales(Retailer,Retailer_ID,Invoice_Date,Region,State,City,Gender_Type,Product_Category,Price_per_Unit,Units_Sold,Total_Sales,Operating_Profit,Operating_Margin,Sales_Method)\n    # \n    전체 state의 총 매출'

## API Call

- 모델 (model): 사용할 모델을 지정합니다. `gpt-3.5-turbo` 와 같은 특정 모델을 선택하여 사용할 수 있습니다.
- 메시지 (message) : 대화의 맥락을 구성하는 메시지 목록. 시스템 메시지와 사용자 메시지를 포함할 수 있으며, 각 메시지는 role 과  content를 가집니다.
- `system` : assistant 의 behavior를 set. instruction을 줌
- `user` : assistant가 반응해야하는 request나 comment를 주는 사용자 역할
- 기타 조정 가능한 파라미터 :
- `max_tokens` : 생성할 텍스트의 최대 길이를 제어. 대화형 모델에서는 응답의 길이를 제한하는 데 유용
- `temperature` : 생성된 응답의 다양성과 창의성을 조정. 낮은 값은 보다 결정적이고 예측 가능한 응답을, 높은 값은 더 창의적이고 다양한 응답을 생성.
- `stop` : 생성을 중지할 특정 문자열 또는 문자열 배열을 지정할 수 있음. 이는 응답을 특정 지점에서 멈추게 하는데 유용

In [153]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an assistant that generates SQL queries based on the given SQLite table definition and a natual language request. The query should start with 'SELECT' and end with a semicolon(;)."},
        {"role": "user", "content": f"A query to answer: {full_prompt}"},
    ],
    max_tokens=200,
    temperature=1.0,
    stop=None
    )

In [154]:
response

ChatCompletion(id='chatcmpl-9cRkyFiXP4lGJwMZ8KEy0PPNIQeJr', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='SELECT State, SUM(Total_Sales) AS Total_Sales\nFROM Sales\nGROUP BY State\nORDER BY Total_Sales DESC;', role='assistant', function_call=None, tool_calls=None))], created=1718950188, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=28, prompt_tokens=141, total_tokens=169))

In [155]:
# 필요한 부분만 추출해보자

# response.choices[0]
# response.choices[0].message
response.choices[0].message.content

'SELECT State, SUM(Total_Sales) AS Total_Sales\nFROM Sales\nGROUP BY State\nORDER BY Total_Sales DESC;'

In [156]:
def handle_response(response):
    query = response.choices[0].message.content.strip()
    
    if not query.upper().startswith("SELECT"):
        query = "SELECT" + query
        
    if not query.endswith(";"):
        query += ";"
        
    return query

In [157]:
print(handle_response(response))

SELECT State, SUM(Total_Sales) AS Total_Sales
FROM Sales
GROUP BY State
ORDER BY Total_Sales DESC;


## Query 를 DB에 입력해보자

In [158]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

2024-06-21 15:09:49,387 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-21 15:09:49,388 INFO sqlalchemy.engine.Engine SELECT State, SUM(Total_Sales) AS Total_Sales
FROM Sales
GROUP BY State
ORDER BY Total_Sales DESC;
2024-06-21 15:09:49,389 INFO sqlalchemy.engine.Engine [generated in 0.00129s] ()
2024-06-21 15:09:49,392 INFO sqlalchemy.engine.Engine ROLLBACK


In [159]:
result.all()

[('New York', 64229039.0),
 ('California', 60174133.0),
 ('Florida', 59283714.0),
 ('Texas', 46359746.0),
 ('South Carolina', 29285637.0),
 ('Washington', 26330718.0),
 ('North Carolina', 23956531.0),
 ('Louisiana', 23750781.0),
 ('Hawaii', 22282457.0),
 ('Virginia', 21575040.0),
 ('Oregon', 21349674.0),
 ('Colorado', 20996536.0),
 ('Nevada', 20858509.0),
 ('New Mexico', 19865016.0),
 ('Idaho', 19276878.0),
 ('Georgia', 18997466.0),
 ('Michigan', 18625433.0),
 ('Wyoming', 18577517.0),
 ('Ohio', 18484583.0),
 ('Tennessee', 18067440.0),
 ('Alabama', 17633424.0),
 ('New Hampshire', 16411667.0),
 ('Arizona', 15782221.0),
 ('Montana', 15710886.0),
 ('Mississippi', 15591709.0),
 ('Alaska', 14753103.0),
 ('Vermont', 14352923.0),
 ('Arkansas', 12639347.0),
 ('Delaware', 12298412.0),
 ('Connecticut', 11573448.0),
 ('Massachusetts', 11076810.0),
 ('West Virginia', 10689160.0),
 ('Oklahoma', 10620734.0),
 ('Pennsylvania', 10357519.0),
 ('Kentucky', 10072848.0),
 ('Kansas', 9972864.0),
 ('New Jers